In [ ]:
import numpy as np
import pandas as pd
import math
import ROOT as root
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from keras import backend as K
from keras.callbacks import History 
from keras.models import load_model
from sklearn.utils import shuffle
from sklearn import preprocessing
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

Loading csv data:

In [ ]:
%cd ../output

In [ ]:
out_dataframe = pd.read_csv('./bxcut_full_muon.csv')
muon_dataframe_test = pd.read_csv('./bxcut_full_test.csv')

Normalize the angle coordinates:

In [ ]:
out_dataframe["1dtPrimitive.phiB"] = out_dataframe["1dtPrimitive.phiB"]/512.
out_dataframe["2dtPrimitive.phiB"] = out_dataframe["2dtPrimitive.phiB"]/512.
out_dataframe["3dtPrimitive.phiB"] = out_dataframe["3dtPrimitive.phiB"]/512.
out_dataframe["4dtPrimitive.phiB"] = out_dataframe["4dtPrimitive.phiB"]/512.

muon_dataframe_test["1dtPrimitive.phiB"] = muon_dataframe_test["1dtPrimitive.phiB"]/512.
muon_dataframe_test["2dtPrimitive.phiB"] = muon_dataframe_test["2dtPrimitive.phiB"]/512.
muon_dataframe_test["3dtPrimitive.phiB"] = muon_dataframe_test["3dtPrimitive.phiB"]/512.
muon_dataframe_test["4dtPrimitive.phiB"] = muon_dataframe_test["4dtPrimitive.phiB"]/512.

Filter the particles with pt less than 200 GeV (just to confirm):

In [ ]:
out_dataframe = out_dataframe[out_dataframe['genParticle.pt'] <= 200]
muon_dataframe_test = muon_dataframe_test[muon_dataframe_test['genParticle.pt'] <= 200]

In [ ]:
out_dataframe

Function for the feature selection

In [ ]:
def preprocess_features(muon_dataframe):
  """Prepares input features from Muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from muon simulations
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = muon_dataframe[
[#'Event',
 'n_Primitive',
 '1dtPrimitive.id_r',
 '2dtPrimitive.id_r',
 '3dtPrimitive.id_r',
 '4dtPrimitive.id_r',
 '1dtPrimitive.id_eta',
 '2dtPrimitive.id_eta',
 '3dtPrimitive.id_eta',
 '4dtPrimitive.id_eta',
 '1dtPrimitive.id_phi',
 '2dtPrimitive.id_phi',
 '3dtPrimitive.id_phi',
 '4dtPrimitive.id_phi',
 '1dtPrimitive.phiB',
 '2dtPrimitive.phiB',
 '3dtPrimitive.phiB',
 '4dtPrimitive.phiB',
 '1dtPrimitive.quality',
 '2dtPrimitive.quality',
 '3dtPrimitive.quality',
 '4dtPrimitive.quality',
 'delta_phi12',
 'delta_phi13',
 'delta_phi14',
 'delta_phi23',
 'delta_phi24',
 'delta_phi34'
  ]]
  processed_features = selected_features.copy()
  return processed_features.astype(np.float32)



And target processing:

In [ ]:
def preprocess_targets(muon_dataframe):
  """
  Prepares target features (i.e., labels) from muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from the Muon data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  max_value = muon_dataframe["genParticle.pt"].max()
  min_value = muon_dataframe["genParticle.pt"].min()
  #output_targets["genParticle.pt"] = (muon_dataframe["genParticle.pt"]-min_value)/(max_value-min_value)
  output_targets["genParticle.pt"] = muon_dataframe["genParticle.pt"]/200.
  return output_targets.astype(np.float32)

Select the datasets for the training and testing:

In [ ]:
X = preprocess_features(out_dataframe)
X_test = preprocess_features(muon_dataframe_test)

In [ ]:
Y = preprocess_targets(out_dataframe)
Y_test = preprocess_targets(muon_dataframe_test)
Y_test = Y_test.fillna(0)

Filter in quality (grouping between 0 and 1)

In [ ]:
X.loc[X["1dtPrimitive.quality"] < 4, '1dtPrimitive.quality'] = 0.0
X.loc[X["1dtPrimitive.quality"] >= 4, '1dtPrimitive.quality'] = 1.0
X.loc[X["2dtPrimitive.quality"] < 4, '2dtPrimitive.quality'] = 0.0
X.loc[X["2dtPrimitive.quality"] >= 4, '2dtPrimitive.quality'] = 1.0
X.loc[X["3dtPrimitive.quality"] < 4, '3dtPrimitive.quality'] = 0.0
X.loc[X["3dtPrimitive.quality"] >= 4, '3dtPrimitive.quality'] = 1.0
X.loc[X["4dtPrimitive.quality"] < 4, '4dtPrimitive.quality'] = 0.0
X.loc[X["4dtPrimitive.quality"] >= 4, '4dtPrimitive.quality'] = 1.0

X_test.loc[X_test["1dtPrimitive.quality"] < 4, '1dtPrimitive.quality'] = 0.0
X_test.loc[X_test["1dtPrimitive.quality"] >= 4, '1dtPrimitive.quality'] = 1.0
X_test.loc[X_test["2dtPrimitive.quality"] < 4, '2dtPrimitive.quality'] = 0.0
X_test.loc[X_test["2dtPrimitive.quality"] >= 4, '2dtPrimitive.quality'] = 1.0
X_test.loc[X_test["3dtPrimitive.quality"] < 4, '3dtPrimitive.quality'] = 0.0
X_test.loc[X_test["3dtPrimitive.quality"] >= 4, '3dtPrimitive.quality'] = 1.0
X_test.loc[X_test["4dtPrimitive.quality"] < 4, '4dtPrimitive.quality'] = 0.0
X_test.loc[X_test["4dtPrimitive.quality"] >= 4, '4dtPrimitive.quality'] = 1.0

Definition of the keras neural network model:

In [ ]:
def baseline_model(size,epochs,optimizer,X_training,y_training,X_validation,y_validation,output_name):
    # create model
    name="RMSE validation"
    name2="RMSE training"
    history = History()
    model = Sequential()
    model.add(Dense(1000, input_dim=27, kernel_initializer='random_normal', activation='sigmoid'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(x_train, y_train,
          batch_size=size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation),callbacks=[history])
    train_predictions = model.predict(X_training)
    predictions = model.predict(X_validation)
    lin_mse = mean_squared_error(y_validation, predictions)
    lin_rmse = np.sqrt(lin_mse)
    lin_mse2 = mean_squared_error(y_training, train_predictions)
    lin_rmse2 = np.sqrt(lin_mse2)
    msg = "%s: %f" % (name, lin_rmse)
    msg2 = "%s: %f" % (name2, lin_rmse2)
    print(msg)
    print(msg2)
    fig,ax = plt.subplots()
    xy=np.vstack([y_validation, predictions])
    z=gaussian_kde(xy)(xy)
    ax.scatter(y_validation, predictions, edgecolors=(0, 0, 0), c=z)
    ax.set_title('Regression model predictions (validation set)')
    ax.set_xlabel('Measured $p_T$ (GeV/c)')
    ax.set_ylabel('Predicted $p_T$ (GeV/c)')
    ax.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=4)
    plt.rc('font', size=20)
    plt.rc('axes', titlesize=18)
    plt.rc('axes', labelsize=18)    
    plt.rc('xtick', labelsize=18)   
    plt.rc('ytick', labelsize=18)  
    plt.rc('legend', fontsize=18)    
    plt.rc('figure', titlesize=18)
    plt.tight_layout()
    plt.savefig('1'+ output_name,format='png',dpi=800)
    fig2,ax2 = plt.subplots()
    ax2.plot(history.history['loss'], label='loss')
    ax2.plot(history.history['val_loss'], label='val_loss')
    ax2.set_title('Training and Validation loss per epoch')
    ax2.set_xlabel('# Epoch')
    ax2.set_ylabel('loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig('2'+ output_name,format='png',dpi=800)
    #plt.show()
    del ax,ax2
    return model

Using train/valid split:

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2)

Model training:

In [ ]:
model = baseline_model(300,250,'Adamax',x_train, y_train, x_valid, y_valid,'Adamax.png')
model.save('./model/my_model1.h5')

In [ ]:
model = baseline_model(300,250,'Adagrad',x_train, y_train, x_valid, y_valid,'Adagrad.png')
model.save('./model/my_model2.h5')

In [ ]:
model = baseline_model(300,250,'RMSprop',x_train, y_train, x_valid, y_valid,'RMSProp.png')
model.save('./model/my_model3.h5')

In [ ]:
model = baseline_model(300,5,'Adam',x_train, y_train, x_valid, y_valid,'Adam.png')
model.save('./model/my_model4.h5')

In [ ]:
model = baseline_model(300,260,'SGD',x_train, y_train, x_valid, y_valid,'SGD.png')
model.save('my_model_all5.h5')

Model testing with indipendent dataset:

In [ ]:
outputFile = root.TFile("prova.root","recreate")
c1 = root.TCanvas("c1","c1");
name="RMSE"
model = load_model('./model/my_model_mu4.h5')
predictions = model.predict(X_test)
lin_mse = mean_squared_error(Y_test, predictions)
n=predictions.size
predictions = predictions.astype('float')
lalla = Y_test.values
lalla = lalla.astype('float')
lin_rmse = np.sqrt(lin_mse)
msg = "%s: %f" % (name, lin_rmse)
print(msg)
fig,ax = plt.subplots()
ax.scatter(Y_test*200, predictions*200, edgecolors=(0, 0, 0))
plot = root.TGraph(n,lalla*200,predictions*200)
plot.SetTitle("Machine Learning model predictions;Generated Muon p_{T} (GeV/c);Predicted Muon p_{T} (GeV/c)")
ax.set_title('Regression model predictions (test)')
ax.set_xlabel('Measured $p_{T}$ (GeV/c)')
ax.set_ylabel('Predicted $p_{T}$ (GeV/c)')
ax.plot([Y.min()*200, Y.max()*200], [Y.min()*200, Y.max()*200], 'k--', lw=4)
plt.rc('font', size=20)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)    
plt.rc('xtick', labelsize=18)   
plt.rc('ytick', labelsize=18)  
plt.rc('legend', fontsize=18)
plt.rc('figure', titlesize=18)
plt.tight_layout()
plt.savefig('validation.png',format='png',dpi=800)
plot.Draw("AP")
c1.Update()
c1.Write("test")

c2 = root.TCanvas("c2","c2");
lallo = X_test["delta_phi12"].values
lallo = lallo.astype('float')
plot2 = root.TGraph(n,lalla*200,lallo)
plot2.SetTitle("Delta phi vs Generated p_{T};Generated Muon p_{T} (GeV/c);delta phi(MB2-MB1)")
plot2.Draw("AP")
c2.Update()
c2.Write("test2")

outputFile.Write()
outputFile.Close()